## Using Komo for IK

In [53]:
import sys
sys.path.append('../../build') #rai/lib')
import numpy as np
import libry as ry

In [35]:
#-- MODEL WORLD configuration, this is the data structure on which you represent
# what you know about the world and compute things (controls, contacts, etc)
C = ry.Config()
#D = C.view() #rather use the ConfiguratioViewer below
C.addFile("../../scenarios/pandasTable.g")

In [36]:
#-- using the viewer, you can view configurations or paths
V = ry.ConfigurationViewer()
V.setConfiguration(C)

In [37]:
# add a new frame to the MODEL configuration
# (Perception will later have to do exactly this: add perceived objects to the model)
obj = C.addFrame("object")

In [38]:
# set frame parameters, associate a shape to the frame, 
obj.setPosition([.8,0,1.5])
obj.setQuaternion([1,0,.5,0])
obj.setShape(ry.ST.capsule, [.2,.02])
obj.setColor([1,0,1])
V.setConfiguration(C)
# V.recopyMeshes(C) #this is rarely necessary, only when you change meshes within C

In [43]:
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.sos, feature=ry.FS.positionDiff, frames=['R_gripperCenter', 'object'])

In [44]:
# Calling the optimizer (True means random initialization/restart)
IK.optimize(True)
IK.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.5, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.118614494036232},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.0004539026017196094},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-object',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.3674662141495262}]

In [57]:
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

In [140]:
# move object
obj.setPosition([.2,.2,1.5])

# copy C into the IK's internal configuration(s)
IK.setConfigurations(C)

# reoptimize
IK.optimize(False) # False indicates: no adding of noise for a random restart
print(IK.getReport())

# grab result
C.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(C) #to update your model display

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.5, 'useSwift': True}, {'order': 0.0, 'type': 'eq', 'feature': 'QuaternionNorms', 'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'eq_sumOfAbs': 0.0}, {'order': 2.0, 'type': 'sos', 'feature': 'qItself#32', 'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'sos_sumOfSqr': 1.04635464428218e-05}, {'order': 0.0, 'type': 'sos', 'feature': 'qItself#32', 'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'sos_sumOfSqr': 0.0015205517268362997}, {'order': 0.0, 'type': 'sos', 'feature': 'Default-0-posDiff-R_gripperCenter-object', 'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'sos_sumOfSqr': 0.0032364231210582043}]


## Path Optimization

In [175]:
obj.setPosition([.8,.2,1.5])

# we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
komo = C.komo_path(1.,20, 1., True) 
komo.addObjective(type=ry.OT.sos, feature=ry.FS.positionDiff, frames=['R_gripperCenter', 'object'], scale=[1e-1])

komo.optimize(True)
komo.getReport()

[{'x_dim': 320, 'T': 20, 'k_order': 2, 'tau': 0.05, 'useSwift': True},
 {'order': 0.0,
  'type': 'eq',
  'feature': 'QuaternionNorms',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'eq_sumOfAbs': 0.0},
 {'order': 2.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.0004133610225569579},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'qItself#32',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.0001518065400876826},
 {'order': 0.0,
  'type': 'sos',
  'feature': 'Default-0-posDiff-R_gripperCenter-object',
  'vars': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'sos_sumOfSqr': 0.07184927382185888}]

In [176]:
V = komo.view()
V.playVideo()